In [7]:
import numpy as np
import pandas as pdb
import lyricsgenius
import json
import requests
import nltk
%pip install stop-words
from stop_words import get_stop_words
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
genius = lyricsgenius.Genius("soOLiPdEPwYDC5ZwsK2Td_jDeG8ZS9FIdNSBnL5dEkfHUsvnjBje47xs6EWG8QJe")

Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alabo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [9]:
r = requests.get('https://raw.githubusercontent.com/priad2020/songs/master/songs.json')
c = r.text
data = json.loads(c)
popList = data.get("pop")
rockList = data.get("rock")
rapList = data.get("rap")

In [10]:
def Songs(List):
    songs = []
    for i in List: 
        songs.append(genius.search_song(i).lyrics)
    return songs


In [11]:
popsongs = Songs(popList)

Searching for "Shape of You"...
Done.
Searching for "thank u, next"...
Done.
Searching for "The Hills"...
Done.
Searching for "God’s Plan"...
Done.
Searching for "Drunk in Love"...
Done.
Searching for "Bohemian Rhapsody"...
Done.
Searching for "Work from Home"...
Done.
Searching for "Too Good at Goodbyes"...
Done.
Searching for "In My Feelings"...
Done.
Searching for "7 rings"...
Done.
Searching for "Hotline Bling"...
Done.
Searching for "Closer"...
Done.
Searching for "Starboy"...
Done.
Searching for "Shallow"...
Done.
Searching for "Formation"...
Done.
Searching for "One Dance"...
Done.
Searching for "Perfect"...
Done.
Searching for "FRIENDS"...
Done.
Searching for "I’m the One"...
Done.
Searching for "Money Trees"...
Done.
Searching for "Hello"...
Done.
Searching for "Bitch, Don’t Kill My Vibe"...
Done.
Searching for "Thrift Shop"...
Done.
Searching for "New Rules"...
Done.
Searching for "Poetic Justice"...
Done.
Searching for "White Iverson"...
Done.
Searching for "Love Yourself"..

In [12]:
rocksongs = Songs(rockList)

Searching for "I Can’t Drive 55"...
Done.
Searching for "Lit Up"...
Done.
Searching for "Frankenstein"...
Done.
Searching for "Carry On Wayward Son"...
Done.
Searching for "Higher"...
Done.
Searching for "I Believe In A Thing Called Love"...
Done.
Searching for "Turn Up The Radio"...
Done.
Searching for "Don’t Tell Me You Love Me"...
Done.
Searching for "Bullet With Butterfly Wings"...
Done.
Searching for "Aqualung"...
Done.
Searching for "Party Hard"...
Done.
Searching for "Would?"...
Done.
Searching for "Seventeen"...
Done.
Searching for "The Beautiful People"...
Done.
Searching for "Slither"...
Done.
Searching for "I Wanna Be Somebody"...
Done.
Searching for "Bring Me To Life"...
Done.
Searching for "Since You Been Gone"...
Done.
Searching for "Heaven And Hell"...
Done.
Searching for "Any Way You Want It"...
Done.
Searching for "Rebel Yell"...
Done.
Searching for "Feel Like Making Love"...
Done.
Searching for "Black Hole Sun"...
Done.
Searching for "Kiss Me Deadly"...
Done.
Searchin

In [13]:
rapsongs = Songs(rapList)

Searching for "Rap God"...
Done.
Searching for "HUMBLE."...
Done.
Searching for "Bad and Boujee"...
Done.
Searching for "Man’s Not Hot"...
Done.
Searching for "God’s Plan"...
Done.
Searching for "SICKO MODE"...
Done.
Searching for "Bodak Yellow"...
Done.
Searching for "XO TOUR Llif3"...
Done.
Searching for "1-800-273-8255"...
Done.
Searching for "In My Feelings"...
Done.
Searching for "SAD!"...
Done.
Searching for "Trap Queen"...
Done.
Searching for "Hotline Bling"...
Done.
Searching for "rockstar"...
Done.
Searching for "Mercy"...
Done.
Searching for "m.A.A.d city"...
Done.
Searching for "Killshot"...
Done.
Searching for "Starboy"...
Done.
Searching for "Jocelyn Flores"...
Done.
Searching for "Holy Grail"...
Done.
Searching for "One Dance"...
Done.
Searching for "Niggas in Paris"...
Done.
Searching for "Panda"...
Done.
Searching for "Look At Me!"...
Done.
Searching for "I’m the One"...
Done.
Searching for "Mask Off"...
Done.
Searching for "Know Yourself"...
Done.
Searching for "Hot Ni

In [31]:
def words(songs):
    words = []
    punctuation = ["!",",", ".","    ",":",";","?","(",")","— ","*","»","…","«","—","[","]"] 
    stopWords = get_stop_words('english')
    stopWords.append("pre-chorus")
    stopWords.append ("verse")
    stopWords.append("post-chorus")
    stopWords.append("chorus")
    for song in songs:
        song = song.lower()
        for sign in punctuation:
             song = song.replace(sign,"")
                
        w = []            
        for word in song.split():
            if not(word in stopWords):
                word = word.replace("'","")
                word = lemmatizer.lemmatize(word)
                w.append(word)
        words.append(w)
                
    return  words
        

In [32]:
popwords = words(popsongs)
rockwords = words(rocksongs)
rapwords = words(rapsongs)


In [33]:
def createDicionary(words):
    dic = {}
    for word in sorted(words):
        if word in  dic.keys():
             dic[word] =  dic[word] + 1
        else:
             dic[word] = 1
    return  dic


In [45]:
length = len(popwords)
lengthLern = int(n * 0.3)
popwords_Learn = popwords[0:lengthLern]
popwords_Test = popwords[lengthLern+1:]

rockwords_Learn = rockwords[0:lengthLern]
rockwords_Test = rockwords[lengthLern+1:]

rapwords_Learn = rapwords[0:lengthLern]
rapwords_Test = rapwords[lengthLern+1:]

22